In [3]:
# load model auxiliary function
from tensorflow.keras import layers
from tensorflow.keras.utils import CustomObjectScope
from keras import backend as K
import tensorflow.keras as keras

def load_model(architecture_file='', weights_file=''):
    def _hard_swish(x):
        return x * K.relu(x + 3.0, max_value=6.0) / 6.0

    def _relu6(x):
        return K.relu(x, max_value=6)

    if '.json' not in architecture_file:
        architecture_file = architecture_file+'.json'

    with open(architecture_file, 'r') as f:
        with CustomObjectScope({'relu6': _relu6,
                                'DepthwiseConv2D': layers.DepthwiseConv2D,
                                '_hard_swish': _hard_swish}):
            model = keras.models.model_from_json(f.read())

    if weights_file != '':
        if '.h5' not in weights_file:
            weights_file = weights_file + '.h5'
        model.load_weights(weights_file)
        print('Load architecture [{}]. Load weights [{}]'.format(architecture_file, weights_file), flush=True)
    else:
        print('Load architecture [{}]'.format(architecture_file), flush=True)

    return model

In [ ]:
from sklearn.metrics._classification import accuracy_score
import keras
import numpy as np
import tensorflow as tf

np.random.seed(12227)

# loading the original cifar10 dataset to extract the mean from the original train set
(x_train, _), (_, _) = keras.datasets.cifar10.load_data()
x_train = x_train.astype('float32') / 255

# extract mean from the original train set
x_train_mean = np.mean(x_train, axis=0)

In [7]:
# loading the adversarial attack cifar10-r
# make sure the cifar-10-r is in the same folder of this notebook, or change the path
# of the np.load function
load = np.load('cifar-10-r.npz')
_, _, X_test, y_test = load['X_train'], load['y_train'], load['X_test'], load['y_test']

In [8]:
# applying the train set mean to the cifar10-r test set
X_test = X_test.astype('float32') / 255
X_test -= x_train_mean

In [ ]:
# load model and evaluate accuracy
model = load_model('path_to_your_model.json',
                   'path_to_your_model.h5')
acc = accuracy_score(np.argmax(y_test, axis=1), np.argmax(model.predict(X_test, verbose=0), axis=1))

print(round(acc, 6))